In [59]:
import requests
import json
import pandas as pd
from functools import reduce

#colorado zip: https://gist.githubusercontent.com/nateirwin/8c318ba1af06fff2eafe/raw/3280fcd5fd5bea88b93d5954e159698ec5f9905e/colorado-cities.csv

In [13]:
#https://taxonomy.nucc.org/?searchTerm=175T00000X&searchButton=search
#Individuals certified to perform peer support services through a training process defined by a government agency, such as the Department of Veterans Affairs or a state mental health department/certification/licensing authority.
taxonomy = '175T00000X'
api_link = 'https://npiregistry.cms.hhs.gov/api/?version=2.1'
taxonomy_text = "Peer Specialist"

In [17]:
skip = 0
finish = 0
data = []

payload = {}
headers = {}

while finish == 0:
    url = f"{api_link}&taxonomy_description={taxonomy_text}&state=CO&limit=200&skip={skip}"

    response = requests.request("GET", url, headers=headers, data=payload)

    response_get = json.loads(response.text)

    data.append(response_get)

    if response_get['result_count'] < 200:
        finish =1
    else:
        skip += 200

    

In [78]:
flatted_data = []
for i in data:
    for j in i['results']:
        flatted_data.append(j)

df = pd.DataFrame.from_records([x for x in flatted_data])

In [79]:
df.head()

,created_epoch,enumeration_type,last_updated_epoch,number,addresses,practiceLocations,basic,taxonomies,identifiers,endpoints,other_names
0,1457632282000,NPI-1,1457632282000,1578924155,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'AARON', 'last_name': 'ABBOTT',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[]
1,1678218037000,NPI-1,1678218037000,1982304085,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'ASIM', 'last_name': 'ABDULLAH'...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[]
2,1444255358000,NPI-1,1444255358000,1447621131,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'EDNA', 'last_name': 'ABRAHAM G...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[]
3,1675682409000,NPI-1,1675682409000,1508562190,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'SCOTT', 'last_name': 'ABRAMS',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[]
4,1549898115000,NPI-1,1549898115000,1679038095,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'SOPHIA', 'last_name': 'ABRAMS'...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],"[{'type': 'Other Name', 'code': '5', 'first_na..."


In [80]:
df.to_csv('npi_data.csv')

In [142]:
df['mailing_city']=[x[0]['city'] for x in df.addresses]
df['zip']=[int(x[0]['postal_code'][:5]) for x in df.addresses]
df['NPI_last_updated']=[x['last_updated'] for x in df.basic]
df['taxonomies_flattened']=[[x['desc'] for x in sublist] for sublist in df.taxonomies]

In [143]:
zips = pd.read_csv('colorado-zip.csv',names=['zip','state','city','lat','lon'],skiprows=1)
zips.head()

,zip,state,city,lat,lon
0,80002,CO,Arvada,39.794533,-105.098402
1,80003,CO,Arvada,39.828572,-105.065549
2,80004,CO,Arvada,39.814066,-105.117710
3,80005,CO,Arvada,39.842189,-105.109719
4,80010,CO,Aurora,39.736788,-104.864618


In [149]:
df_with_location = df.merge(zips,on='zip')
df_with_location.head()

,created_epoch,enumeration_type,last_updated_epoch,number,addresses,practiceLocations,basic,taxonomies,identifiers,endpoints,...,mailing_city,NPI_last_updated,taxonomies_flattened,taxonomies_len,mailing_zip,zip,state,city,lat,lon
0,1457632282000,NPI-1,1457632282000,1578924155,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'AARON', 'last_name': 'ABBOTT',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],...,CANON CITY,2016-03-10,[Peer Specialist],1,81212,81212,CO,Canon City,38.445074,-105.217829
1,1543257694000,NPI-1,1543257694000,1235609975,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'AMY', 'last_name': 'BERGANT', ...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],...,CANON CITY,2018-11-26,[Peer Specialist],1,81212,81212,CO,Canon City,38.445074,-105.217829
2,1459377920000,NPI-1,1459377920000,1962865766,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'DUSTY', 'last_name': 'DASUGO',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],...,CANON CITY,2016-03-30,[Peer Specialist],1,81212,81212,CO,Canon City,38.445074,-105.217829
3,1461081987000,NPI-1,1461081987000,1730533415,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'LUZ', 'last_name': 'DIAZ-PORTE...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],...,CANON CITY,2016-04-19,[Peer Specialist],1,81212,81212,CO,Canon City,38.445074,-105.217829
4,1543339285000,NPI-1,1543339285000,1669942231,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'DEAN', 'last_name': 'GARCIA', ...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],...,CANON CITY,2018-11-27,[Peer Specialist],1,81212,81212,CO,Canon City,38.445074,-105.217829


In [150]:
df_with_location.to_csv('npi_data.csv')

In [126]:
df.head()

,created_epoch,enumeration_type,last_updated_epoch,number,addresses,practiceLocations,basic,taxonomies,identifiers,endpoints,other_names,mailing_city,NPI_last_updated,taxonomies_flattened,taxonomies_len,mailing_zip
0,1457632282000,NPI-1,1457632282000,1578924155,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'AARON', 'last_name': 'ABBOTT',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[],CANON CITY,2016-03-10,[Peer Specialist],1,81212
1,1678218037000,NPI-1,1678218037000,1982304085,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'ASIM', 'last_name': 'ABDULLAH'...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[],AURORA,2023-03-07,[Peer Specialist],1,80011
2,1444255358000,NPI-1,1444255358000,1447621131,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'EDNA', 'last_name': 'ABRAHAM G...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[],ENGLEWOOD,2015-10-07,[Peer Specialist],1,80112
3,1675682409000,NPI-1,1675682409000,1508562190,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'SCOTT', 'last_name': 'ABRAMS',...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],[],WHEAT RIDGE,2023-02-06,[Peer Specialist],1,80033
4,1549898115000,NPI-1,1549898115000,1679038095,"[{'country_code': 'US', 'country_name': 'Unite...",[],"{'first_name': 'SOPHIA', 'last_name': 'ABRAMS'...","[{'code': '175T00000X', 'taxonomy_group': '', ...",[],[],"[{'type': 'Other Name', 'code': '5', 'first_na...",GRAND JUNCTION,2019-02-11,[Peer Specialist],1,81506


In [159]:
mapping = df_with_location[['enumeration_type','lat','lon','number']].groupby(['enumeration_type','lat','lon']).count().reset_index()
mapping.head()

,enumeration_type,lat,lon,number
0,NPI-1,37.050570,-105.575625,1
1,NPI-1,37.470274,-105.878602,8
2,NPI-1,38.284277,-104.623370,1
3,NPI-1,38.287876,-104.584828,41
4,NPI-1,38.445074,-105.217829,14


In [158]:
mapping['number']

0      1
1      8
2      1
3     42
4     14
      ..
58     1
59     3
60    40
61     7
62     4
Name: number, Length: 63, dtype: int64